In [1]:
import pandas
import numpy as np
import tensorflow as tf
from keras.utils import np_utils
# CNN related
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.recurrent import GRU
#from keras.layers.recurrent import LSTM, GRU
from keras.layers import Conv1D, MaxPooling1D, AtrousConvolution1D, RepeatVector
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from keras.layers.wrappers import Bidirectional
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import *
from keras.optimizers import RMSprop, Adam, SGD, Nadam
from keras.initializers import *
import matplotlib.pyplot as plt

from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE # doctest: +NORMALIZE_WHITESPACE
from imblearn.over_sampling import RandomOverSampler # doctest: +NORMALIZE_WHITESPACE
from imblearn.over_sampling import SVMSMOTE
from imblearn.over_sampling import KMeansSMOTE
from imblearn.over_sampling import BorderlineSMOTE # doctest: +NORMALIZE_WHITESPACE
from imblearn.combine import SMOTETomek # doctest: +NORMALIZE_WHITESPACE

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from keras import backend as K


##################
# AUC for a binary classifier
def auc(y_true, y_pred):
    ptas = tf.stack([binary_PTA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)
    pfas = tf.stack([binary_PFA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)
    pfas = tf.concat([tf.ones((1,)) ,pfas],axis=0)
    binSizes = -(pfas[1:]-pfas[:-1])
    s = ptas*binSizes
    return K.sum(s, axis=0)
    
##################
# PFA, prob false alert for binary classifier
def binary_PFA(y_true, y_pred, threshold=K.variable(value=0.5)):
    y_pred = K.cast(y_pred >= threshold, 'float32')
    # N = total number of negative labels
    N = K.sum(1 - y_true)
    # FP = total number of false alerts, alerts from the negative class labels
    FP = K.sum(y_pred - y_pred * y_true)
    #print(FP/N)
    return FP/N

##################
# P_TA prob true alerts for binary classifier
def binary_PTA(y_true, y_pred, threshold=K.variable(value=0.5)):
    y_pred = K.cast(y_pred >= threshold, 'float32')
    # P = total number of positive labels
    P = K.sum(y_true)
    # TP = total number of correct alerts, alerts from the positive class labels
    TP = K.sum(y_pred * y_true)
    return TP/P

class ecgcnn: 
    np.set_printoptions(threshold=np.inf)
    def __init__(self, base_path, stkid, infosize, onehotsize ):
        self.stkid = stkid
        fname = base_path + stkid + '.csv'
        self.INFOSIZE = infosize
        self.ONEHOTSIZE = onehotsize
        self.load_data(fname)
        self.EMB_SIZE = self.INFOSIZE+self.ONEHOTSIZE
        #self.ONEHOTSIZE = onehotsize
        

##################
    def load_data(self, fname):
        # load csv data
        mat = pandas.read_csv(fname, sep=",", header=0, error_bad_lines=False).as_matrix()

        self.info = []
        self.sid=[]
        for i in range(self.INFOSIZE):
            self.info.append(self.matrix_col(mat, i))   
        self.sid.append(self.matrix_col(mat,9))
       # print(self.sid) 
        #x = self.find_category(self.sid)
        #print(x)
##################        
    '''
    def find_category0(self, sid):
        print(sid[0][0])
        y = list(map(int, sid[3:-1])) 
        #y = map(eval,sid[3:-1])
        print(y)    
        for n in range(len(sid)):
            sidnum = [0]*51 
            if ( y == 11 ):
                print("456") 
                sidnum [n]=sidnum [n]+1
                print(sidnum)
                return  sidnum
    '''
    def find_category(self, sid):
        result = []
        for sidstr in sid[0]:
            index = int(sidstr.replace('sid', ''))
            result.append(index)
            
        return result
        
    '''           
            sidnum = [0]*51
            #print(index)
            sidnum[index-1] = 1
            #print(sidnum)
            result.append(sidnum)
           
            return result
    ''' 
######################################################
    def matrix_col(self, matrix, i):
        return [row[i] for row in matrix]    
    
######################################################
    def normalize(self, data):
        #return (np.array(data)-np.mean(data))/np.std(data) 
        std = np.std(data)
        if (std == 0):
            data2 = np.zeros(len(data))
            return data2
        return (np.array(data)-np.mean(data))/std
    
######################################################
    
    def backtest_training_data(self,num,s):
        x_train0, y_train_onehot = [], []  
        x_train1, x_test1, y_train1, y_test1 = [], [], [], []
        # normalize
        nvals = []
        for cval in self.info:
            nvals.append(self.normalize(cval))
        
        # training preparation            
        l2t = nvals
        x_train0 = np.column_stack(l2t)
        y_train_onehot = self.find_category(self.sid)

        x_train = []
        for rec in x_train0:
            x_train.append(rec)
        
        x_train = np.array(x_train)
        #y_train_onehot = np_utils.to_categorical(y_train)
        y_train_onehot = np.array(y_train_onehot)

        if num==0:
            n = 40
        if num==1:
            n = 37
        if num==2:
            n = 16
        if num==3:
            n = 84
        x_train1, x_test1, y_train1, y_test1 = train_test_split(x_train, y_train_onehot, test_size=0.2, random_state=None)
        
        
        if s==0:
            x_res, y_res = x_train1, y_train1
        else:    
            if s==1:
                sm = SMOTE(random_state=0)
            if s==2:
                sm = SVMSMOTE(random_state=0)
            if s==3:
                sm = RandomOverSampler(random_state=0)
            if s==4:
                sm = BorderlineSMOTE(random_state=0) 
            if s==5:
                sm = SMOTETomek(random_state=0)
            #sm = KMeansSMOTE(random_state=0)  #X need 25 samples
            
            x_res, y_res = sm.fit_resample(x_train1, y_train1)
            
            #print(x_train)
            #print('Resampled dataset shape %s' % Counter(y_res))

        x_train_new = []
        for rec in x_res:
            x_train_new.append([rec])
            #x_train_new.append(np.reshape(rec,(-1,self.EMB_SIZE)))
        x_test_new = []
        for rec1 in x_test1:
            x_test_new.append([rec1])
                
        x_train_new = np.array(x_train_new)  
        x_test_new = np.array(x_test_new)
        #print(x_train_new)
        
        y_train_onehot_new = []
        y_test_new = [] 
        y_test_new1 = [] 
        
        for sid in y_res:
            index = int(sid)
            sidnum = [0]*n
           
            #print(index)
            sidnum[index-1] = 1
            #print(index)
            #print(sidnum)
            y_train_onehot_new.append(sidnum)  
        for sidstr in y_test1:
            index = int(sidstr)
            sidnum1 = [0]*n
            
            #print(index)
            sidnum1[index-1] = 1
            #print(sidnum)
            y_test_new.append(sidnum1) 
            y_test_new1.append([sidnum1])

        return x_train_new, y_train_onehot_new, x_test_new, y_test_new, y_test_new1

######################################################
    def build_model(self,num):
        return self.simple_model(num)

######################################################
    def simple_model(self,num):
        model = Sequential()
        model.add(Conv1D(input_shape=(1, self.EMB_SIZE), filters=128, kernel_size=16, padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU())
        model.add(Dropout(0.5))

        model.add(Conv1D(filters=64, kernel_size=8, padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU())
        model.add(Dropout(0.5))
       
        model.add(Flatten())
        model.add(Dense(128))
        model.add(BatchNormalization())
        model.add(LeakyReLU())
        if num==0:
            model.add(Dense(40))
        if num==1:
            model.add(Dense(37))
        if num==2:
            model.add(Dense(16))
        if num==3:
            model.add(Dense(84))
            
        model.add(Activation('softmax'))

        opt = Nadam(lr=0.002)
        
        #model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=[auc])
        model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
        #print(model.summary())
        return model

######################################################
    def train_model(self, model, x_train, y_train_onehot, x_test1, y_test1 ):
        model_name = 'cnn.'+self.stkid+'.model'
        input_shape = x_train[0].shape
        #reduce_lr = ReduceLROnPlateau(monitor='roc_auc_val', factor=0.8, patience=30, min_lr=0.000001, verbose=2)
        reduce_lr = ReduceLROnPlateau(monitor='roc_accuracy', factor=0.8, patience=30, min_lr=0.000001, verbose=2)
        checkpointer = ModelCheckpoint(filepath=model_name, verbose=0, save_best_only=True)
        '''
        train_history = model.fit(np.array(x_train),np.array(y_train_onehot), epochs = 200, batch_size = 10, verbose=2, \
                                  validation_data=(np.array(x_test1), np.array(y_test1)), callbacks=[reduce_lr, checkpointer], shuffle=True)
        '''
        train_history = model.fit(np.array(x_train),np.array(y_train_onehot), epochs = 200, batch_size = 100, verbose=0, \
                                   callbacks=[reduce_lr, checkpointer], shuffle=True)
        
        #show_train_history(train_history, 'accuracy', 'val_accuracy')
        #show_train_history(train_history, 'auc', 'val_auc')
        
        #show_train_history(train_history, 'loss', 'val_loss')
        return train_history

######################################################
    def do_backtest(self,num,s):

        # cut data 
        x_train, y_train_onehot, x_test1, y_test1, y_test2 = self.backtest_training_data(num,s)
        # build model 
        model = self.build_model(num)
        train_history = self.train_model(model, x_train, y_train_onehot, x_test1, y_test1 )
        
        if num==0:
            n = 40
        if num==1:
            n = 37
        if num==2:
            n = 16
        if num==3:
            n = 84
            
        count = 0
        total = 0
        #print(x_test1)
        #print(np.array(y_test2))
        #print("x : "+str(x_test1.shape)+" / y : "+str(np.array(y_test2).shape))
        for x_test, y_test in zip(x_test1, np.array(y_test2)):
            
            prediction = model.predict_classes(np.array([x_test]),verbose=0)
            #print(proba[0], proba[1])
            #print(datestr, prediction[0], y_test_onehot[0], scores[1], last_close)
            
      
            index = int(prediction)+1
            #print("prediction: "+str(index))
        
            sidnum = [0]*n
            #print(index)
            sidnum[index-1] = 1
            sidnum_x = np.array(sidnum)
            
            total += 1
            
            if (y_test[0][index-1] == sidnum_x[index-1]):
                count += 1
        if (total == 0):
            return 0, 0
        
    
        #print(total)
        #print(count)
        return count, total
        
######################################################
"""
def show_train_history(train_history, train, validation):
    plt.plot(train_history.history[train])
    #plt.plot(train_history.history[validation])
    plt.title('Train History')
    plt.ylabel(train)
    plt.xlabel('Epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
"""
def main():
    print("start")
    stkid = 'ALL_fiducial_2' 
    TID = 'fiducial'
    DA = ["AUSAIC Data","MIT-BIH Arrhythmia Data","MIT-BIH Normal Sinus Rhythm Data","QT Data"]
    SMO = ["Original","SMOTE","SVMSMOTE","ROS","BorderlineSMOTE","SMOTETomek"]
    ADD = ["/home/keg/桌面/ecg_data/AUSAIC Data/fiducial/",
          "/home/keg/桌面/ecg_data/MIT-BIH Arrhythmia Data/fiducial/",
          "/home/keg/桌面/ecg_data/MIT-BIH Normal Sinus Rhythm Data/fiducial/",
          "/home/keg/桌面/ecg_data/QT Data/fiducial/"]
    NUMB = [0, 1, 2, 3]
    NUMBE = [0, 1, 2, 3, 4, 5]
    info_size = 8
    onehot_size = 0
    file = open('/home/keg/桌面/ecg_data/fiducial_accuracy.csv', 'w')
    file.write('Method,DATA,Ture/Total,accuracy'+'\n')
    for num in NUMB:
        base_path = ADD[num]
        #S=0  #SMOTE /S=1  #SVMSMOTE /S=2  #ROS  /S=3  #BorderlineSMOTE  /S=4  #SMOTETomek
        for S in NUMBE:
            for time in range(5):
                ecg = ecgcnn(base_path, stkid, info_size, onehot_size )
                count, total = ecg.do_backtest(num,S)
                hitrate = 0.0
                if (total > 0):
                    hitrate = float(count)/total
                    print(SMO[S]+","+DA[num]+",("+str(count)+"/"+str(total)+"),"+str(hitrate))
                    plan =SMO[S]+","+DA[num]+",("+str(count)+"/"+str(total)+"),"+str(hitrate)+'\n'
                    #file = open('/home/keg/桌面/ecg_data/fiducial_accuracy.txt', 'w')
                    file.write(plan)
    file.close()
    print("finish")
    #xt, yt = ecg.backtest_training_data()
    #print(xt)
    #print(yt[-1])
    #for stkid in stklist:    
    #    model = ecgcnn(base_path, stkid,info_size)
                      
if (__name__ == "__main__"):
    main()


/home/keg/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/keg/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/keg/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/keg/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

start


/home/keg/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:80: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
W0114 12:57:43.970103 139947361502976 deprecation_wrapper.py:119] From /home/keg/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

/home/keg/anaconda3/lib/python3.6/site-packages/keras/callbacks/callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `roc_accuracy` which is not available. Available metrics are: loss,accuracy,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
/home/keg/anaconda3/lib/python3.6/site-packages/keras/callbacks/callbacks.py:707: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


Original,AUSAIC Data,(767/871),0.8805970149253731


/home/keg/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:80: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/home/keg/anaconda3/lib/python3.6/site-packages/keras/callbacks/callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `roc_accuracy` which is not available. Available metrics are: loss,accuracy,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
/home/keg/anaconda3/lib/python3.6/site-packages/keras/callbacks/callbacks.py:707: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


...

SMOTETomek,QT Data,(1562/1610),0.9701863354037267


/home/keg/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:80: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/home/keg/anaconda3/lib/python3.6/site-packages/keras/callbacks/callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `roc_accuracy` which is not available. Available metrics are: loss,accuracy,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
/home/keg/anaconda3/lib/python3.6/site-packages/keras/callbacks/callbacks.py:707: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


SMOTETomek,QT Data,(1582/1610),0.9826086956521739


/home/keg/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:80: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/home/keg/anaconda3/lib/python3.6/site-packages/keras/callbacks/callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `roc_accuracy` which is not available. Available metrics are: loss,accuracy,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
/home/keg/anaconda3/lib/python3.6/site-packages/keras/callbacks/callbacks.py:707: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


SMOTETomek,QT Data,(1573/1610),0.9770186335403727
finish
